In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load the preprocessed data from the CSV file
preprocessed_data_file = 'datasets/preprocessed_data.csv'
data = pd.read_csv(preprocessed_data_file)

In [3]:
data.shape

(63136, 12)

In [5]:
# Tokenization
tokenizer = Tokenizer()
data['Body/Text'] = data['Body/Text'].fillna('')  # Replace missing values with empty strings
tokenizer.fit_on_texts(data['Body/Text'])

In [6]:
# Create Input-Output Pairs
inputs = tokenizer.texts_to_sequences(data['Body/Text'])
outputs = tokenizer.texts_to_sequences(data['Body/Text'])

In [7]:
# Numerical Encoding
vocab_size = len(tokenizer.word_index) + 1
max_seq_length = 50  # Maximum sequence length for padding/truncation

In [8]:
# Padding and Truncation
inputs = pad_sequences(inputs, maxlen=max_seq_length, padding='post', truncating='post')
outputs = pad_sequences(outputs, maxlen=max_seq_length, padding='post', truncating='post')

In [9]:
# Data Split
train_inputs, val_test_inputs, train_outputs, val_test_outputs = train_test_split(inputs, outputs, test_size=0.2, random_state=42)
val_inputs, test_inputs, val_outputs, test_outputs = train_test_split(val_test_inputs, val_test_outputs, test_size=0.5, random_state=42)

In [10]:
# Print the shapes of the data arrays
print("Train Inputs shape:", train_inputs.shape)
print("Train Outputs shape:", train_outputs.shape)
print("Validation Inputs shape:", val_inputs.shape)
print("Validation Outputs shape:", val_outputs.shape)
print("Test Inputs shape:", test_inputs.shape)
print("Test Outputs shape:", test_outputs.shape)

Train Inputs shape: (50508, 50)
Train Outputs shape: (50508, 50)
Validation Inputs shape: (6314, 50)
Validation Outputs shape: (6314, 50)
Test Inputs shape: (6314, 50)
Test Outputs shape: (6314, 50)


In [32]:
# Create Input-Output Pairs
sequences = tokenizer.texts_to_sequences(data['Body/Text'])

# Split sequences into input and output
inputs = sequences[:-1]
outputs = sequences[1:]

# Padding and Truncation
inputs = pad_sequences(inputs, maxlen=max_seq_length, padding='post', truncating='post')
outputs = pad_sequences(outputs, maxlen=max_seq_length, padding='post', truncating='post')

# Data Split
train_inputs, val_test_inputs, train_outputs, val_test_outputs = train_test_split(inputs, outputs, test_size=0.2, random_state=42)
val_inputs, test_inputs, val_outputs, test_outputs = train_test_split(val_test_inputs, val_test_outputs, test_size=0.5, random_state=42)

# Print the shapes of the data arrays
print("Train Inputs shape:", train_inputs.shape)
print("Train Outputs shape:", train_outputs.shape)
print("Validation Inputs shape:", val_inputs.shape)
print("Validation Outputs shape:", val_outputs.shape)
print("Test Inputs shape:", test_inputs.shape)
print("Test Outputs shape:", test_outputs.shape)


Train Inputs shape: (50508, 50)
Train Outputs shape: (50508, 50)
Validation Inputs shape: (6313, 50)
Validation Outputs shape: (6313, 50)
Test Inputs shape: (6314, 50)
Test Outputs shape: (6314, 50)


In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define the model architecture
model = Sequential()
model.add(Embedding(vocab_size, 256, input_length=max_seq_length))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))
model.add(Dense(vocab_size, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Train the model
model.fit(train_inputs, train_outputs, validation_data=(val_inputs, val_outputs), batch_size=32, epochs=10)

# Generate responses
def generate_response(input_text):
    input_sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = pad_sequences(input_sequence, maxlen=max_seq_length, padding='post', truncating='post')
    predicted_sequence = model.predict(padded_sequence)
    predicted_token_id = tf.argmax(predicted_sequence, axis=-1).numpy()[0]
    response = tokenizer.sequences_to_texts([[predicted_token_id]])[0]
    return response

# Test the chatbot
user_input = "Hello"
response = generate_response(user_input)
print(f"Chatbot: {response}")


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\Users\neele\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\neele\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\neele\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\neele\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\neele\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\neele\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\neele\anaconda